Çizgiden geçen araç sayısını bulma.

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
model = YOLO("yolov8m.pt")

labels=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
'hair drier', 'toothbrush']

In [4]:
video_path = 'C:/Users/reybe/Desktop/staj/YOLOv8 foto/trafik videosu2.mp4'

kamera = cv2.VideoCapture(video_path)

ret, kare = kamera.read()

#Okunan karenin boyutlarını (Yükseklik, Genişlik, Kanal sayısı) alıyoruz.
#Kanal sayısınıkullanmayacağımız için _ koyarak atlıyoruz.
H, W, _ = kare.shape

#Sayma çizgisinin Y koordinatını videonun dikey olarak tam ortası olarak belirliyoruz
Sayma_cizgisi= int(H / 2)

font = cv2.FONT_HERSHEY_SIMPLEX

sayac = 0  #Çizgiyi geçen toplam nesne sayısını tutacak sayaç.
counted_ids = set() #Çizgiyi geçen nesnelerin ID'lerini depolayacak küme (bir ID'yi tekrar saymamak için).

#Videoyu tekrar başa sarıyoruz, çünkü boyutları almak için ilk kareyi okuduk.
kamera.set(cv2.CAP_PROP_POS_FRAMES, 0)

#Video penceresini oluşturuyoruz ve manuel olarak yeniden boyutlandırılabilir yapıyoruz
cv2.namedWindow("YOLOv8 Nesne Sayma", cv2.WINDOW_NORMAL)

while True:

    ret, kare = kamera.read()
    #Belirlediğimiz sanal sayma çizgisini videodaki her karenin üzerine çizdiriyoruz.
    #Parametreler: sırasıyla çizilecek resim, başlangıç noktası(x,y), bitiş noktası(x,y), renk(BGR), kalınlık.
    cv2.line(kare, (0, Sayma_cizgisi), (W, Sayma_cizgisi), (0, 255, 0), 2)

    #classes=[2, 5, 7]= Sadece sınıf ID'si 2 (car), 5 (bus), 7 (truck) olan nesneleri takip et
    results = model.track(kare, persist=True, verbose=False, classes=[2, 5, 7])

    # Eğer model o karede takip edilecek bir nesne bulduysa yani ID atadıysa devam edecek if.
    if results[0].boxes.id is not None:

        #.cls: Tespit edilen nesnenin sınıf ID'si unutmayalım.
        #.cpu().numpy().astype(int): Verileri GPU'dan CPU'ya aktarıp NumPy dizisine çevirir ve tamsayı yapar.
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        cls_ids = results[0].boxes.cls.cpu().numpy().astype(int)

        #Tespit edilen her bir nesnenin kutusu, ID'si ve sınıf ID'si üzerinde döngüye girelim.
        for box, id, cls_id in zip(boxes, ids, cls_ids):
       
            x1, y1, x2, y2 = box
            name = labels[cls_id]

            #Nesnenin alt kenarının Y koordinatını (y2) sayma için referans alıyoruz.
            #rastgele aldım.
            center_y = y2

            if center_y < Sayma_cizgisi and id not in counted_ids:
                sayac += 1 
                counted_ids.add(id)  #Bu ID'yi sayıldı kümesine ekleyelim bir daha sayılmaması için.

                #Çizgiyi yeni geçen nesnenin kutusunu kırmızı renkte çizdirelim.
                cv2.rectangle(kare, (x1, y1), (x2, y2), (0, 0, 255), 2)
                #Kutu üzerine sınıf adı, ID'si ve SAYILDI metnini KIRMIZI renkte yazdır.
                cv2.putText(kare, f"{name} ID:{id} SAYILDI", (x1, y1 - 10), font, 0.5, (0, 0, 255), 2)
            else:
                #Henüz çizgiyi geçmemiş veya zaten sayılmış nesnelerin kutusunu MAVİ renkte çiz.
                cv2.rectangle(kare, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #Kutu üzerine sadece sınıf adını ve ID'sini mav, renkte yaz
                cv2.putText(kare, f"{name} ID:{id}", (x1, y1 - 10), font, 0.5, (255, 0, 0), 2)

    #Toplam sayacın güncel değerini karenin sol üst köşesine yazdır.
    cv2.putText(kare, f"Cizgiyi Gecen Arac Sayisi: {sayac}", (10, 30), font, 1, (0, 255, 0), 2)

    #İşlenmiş (üzerine çizimler yapılmış) kareyi ekranda göster
    cv2.imshow("YOLOv8 Nesne Sayma", kare)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("'q' tuşuna basıldı, çıkılıyor...")
        break

kamera.release()
cv2.destroyAllWindows()

'q' tuşuna basıldı, çıkılıyor...


Nesnenin ilerlediği yol.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict #ID'lerin yollarını saklamak için koleksiyon kullanacağım.
model = YOLO("yolov8m.pt")
labels=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
'hair drier', 'toothbrush']

video_path = 'C:/Users/reybe/Desktop/staj/YOLOv8 foto/kalabalık insan.mp4'

kamera = cv2.VideoCapture(video_path)
ret, kare = kamera.read()
H, W, _ = kare.shape
Sayma_cizgisi = int(H / 2)
font = cv2.FONT_HERSHEY_SIMPLEX
sayac = 0
counted_ids = set()
track_history = defaultdict(lambda: [])

kamera.set(cv2.CAP_PROP_POS_FRAMES, 0)

cv2.namedWindow("YOLOv8 İnsan Sayma ve Takibi", cv2.WINDOW_NORMAL)

while True:
    ret, kare = kamera.read()
    cv2.line(kare, (0, Sayma_cizgisi), (W, Sayma_cizgisi), (0, 255, 0), 2)


    results = model.track(kare, persist=True, verbose=False, classes=[0]) # Sınıf ID 0 = 'person'

    if results[0].boxes is not None and results[0].boxes.id is not None:
            
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)
            
            for box, id in zip(boxes, ids):
                x1, y1, x2, y2 = box
                name = 'person'

                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                current_point = (center_x, center_y)
                track = track_history[id]
                track.append(current_point)
                if len(track) > 30:
                    track.pop(0)

                line_color = (0, 255, 255)
                line_thickness = 3
                points = None
                if len(track) >= 2:
                    points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                    cv2.polylines(kare, [points], isClosed=False, color=line_color, thickness=line_thickness)

                if len(track) >= 2:
                    prev_point_y = track[-2][1]
                    current_point_y = track[-1][1]

                    crossed_up = (prev_point_y >= Sayma_cizgisi and current_point_y < Sayma_cizgisi)
                    crossed_down = (prev_point_y <= Sayma_cizgisi and current_point_y > Sayma_cizgisi)

                    if (crossed_up or crossed_down) and id not in counted_ids:
                        sayac += 1
                        counted_ids.add(id)
                        cv2.rectangle(kare, (x1, y1), (x2, y2), (0, 0, 255), 2)
                        cv2.putText(kare, f"ID:{id} SAYILDI", (x1, y1 - 10), font, 0.5, (0, 0, 255), 2)
                        if points is not None: # points tanımlıysa çiz
                           cv2.polylines(kare, [points], isClosed=False, color=(0, 0, 255), thickness=line_thickness + 1)
                    else:
                        cv2.rectangle(kare, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.putText(kare, f"ID:{id}", (x1, y1 - 10), font, 0.5, (255, 0, 0), 2)
                else:
                     cv2.rectangle(kare, (x1, y1), (x2, y2), (255, 0, 0), 2)
                     cv2.putText(kare, f"ID:{id}", (x1, y1 - 10), font, 0.5, (255, 0, 0), 2)


    cv2.putText(kare, f"Cizgiyi Gecen Insan Sayisi: {sayac}", (10, 30), font, 1, (0, 255, 0), 2)


    cv2.imshow("YOLOv8 Insan Sayma ve Takibi", kare)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("'q' tuşuna basıldı, çıkılıyor...")
        break

print("Döngü bitti, kaynaklar serbest bırakılıyor.")
kamera.release()
cv2.destroyAllWindows()
for i in range(5):
    cv2.waitKey(1)

print("Program sonlandı.")